## Input Data

In [62]:
#librarynya blm lengkap ya masih pake yang sekiranya perlu dulu
#ntar tambahin aja
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import plotly.express as px
import zipfile
import tensorflow as tf
import random
import urllib.request
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from tensorflow import keras
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense, Input, Layer
from os import listdir

### read and process data

In [58]:
def download_and_extract_zip(url, extract_to, local_zip):
    urllib.request.urlretrieve(url, local_zip)
    with zipfile.ZipFile(local_zip, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    os.remove(local_zip)
#path = 'https://github.com/yohanasidabutar623/ML-Part/raw/1877da255490dfe88765eab806046305622b4bce/Data/images'
# Data Images
data_url_1 = 'https://github.com/yohanasidabutar623/ML-Part/raw/1877da255490dfe88765eab806046305622b4bce/Data/images.csv.zip'
local_file_images = 'images.csv.zip'
extract_to_images = 'images_data'
download_and_extract_zip(data_url_1, extract_to_images, local_file_images)
csv_file_path_images = os.path.join(extract_to_images, 'images.csv')
images = pd.read_csv(csv_file_path_images, on_bad_lines='skip')
images['id'] = images['filename'].apply(lambda x: x.replace(".jpg","")).astype(int)
print(images.head())

# Data Styles
data_url_2 = 'https://github.com/yohanasidabutar623/ML-Part/raw/1877da255490dfe88765eab806046305622b4bce/Data/styles.csv.zip'
local_file_styles = 'styles.csv.zip'
extract_to_styles = 'styles_data'
download_and_extract_zip(data_url_2, extract_to_styles, local_file_styles)
csv_file_path_styles = os.path.join(extract_to_styles, 'styles.csv')
styles = pd.read_csv(csv_file_path_styles, on_bad_lines='skip')
styles['filename'] = styles['id'].astype(str) + '.jpg'
styles = styles.dropna()
print(styles.head())

    filename                                               link     id
0  15970.jpg  http://assets.myntassets.com/v1/images/style/p...  15970
1  39386.jpg  http://assets.myntassets.com/v1/images/style/p...  39386
2  59263.jpg  http://assets.myntassets.com/v1/images/style/p...  59263
3  21379.jpg  http://assets.myntassets.com/v1/images/style/p...  21379
4  53759.jpg  http://assets.myntassets.com/v1/images/style/p...  53759
      id gender masterCategory subCategory  articleType baseColour  season  \
0  15970    Men        Apparel     Topwear       Shirts  Navy Blue    Fall   
1  39386    Men        Apparel  Bottomwear        Jeans       Blue  Summer   
2  59263  Women    Accessories     Watches      Watches     Silver  Winter   
3  21379    Men        Apparel  Bottomwear  Track Pants      Black    Fall   
4  53759    Men        Apparel     Topwear      Tshirts       Grey  Summer   

     year   usage                             productDisplayName   filename  
0  2011.0  Casual          

In [ ]:
#Data Styles
styles['filename'] = styles['id'].astype(str) + '.jpg'
styles = styles.dropna()
styles.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,filename
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,39386.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,59263.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,53759.jpg


In [ ]:
#ubah file nama agar sama dengan id
#baru di merge
images['id'] = images['filename'].apply(lambda x: x.replace(".jpg","")).astype(int)
images

,filename,link,id
0,15970.jpg,http://assets.myntassets.com/v1/images/style/p...,15970
1,39386.jpg,http://assets.myntassets.com/v1/images/style/p...,39386
2,59263.jpg,http://assets.myntassets.com/v1/images/style/p...,59263
3,21379.jpg,http://assets.myntassets.com/v1/images/style/p...,21379
4,53759.jpg,http://assets.myntassets.com/v1/images/style/p...,53759
...,...,...,...
44441,17036.jpg,http://assets.myntassets.com/v1/images/style/p...,17036
44442,6461.jpg,http://assets.myntassets.com/v1/images/style/p...,6461
44443,18842.jpg,http://assets.myntassets.com/v1/images/style/p...,18842
44444,46694.jpg,http://assets.myntassets.com/v1/images/style/p...,46694


In [ ]:
#merge
merge_data = styles.merge(images, on='id', how='left')
print(merge_data.head())

      id gender masterCategory subCategory  articleType baseColour  season  \
0  15970    Men        Apparel     Topwear       Shirts  Navy Blue    Fall   
1  39386    Men        Apparel  Bottomwear        Jeans       Blue  Summer   
2  59263  Women    Accessories     Watches      Watches     Silver  Winter   
3  21379    Men        Apparel  Bottomwear  Track Pants      Black    Fall   
4  53759    Men        Apparel     Topwear      Tshirts       Grey  Summer   

     year   usage                             productDisplayName filename_x  \
0  2011.0  Casual               Turtle Check Men Navy Blue Shirt  15970.jpg   
1  2012.0  Casual             Peter England Men Party Blue Jeans  39386.jpg   
2  2016.0  Casual                       Titan Women Silver Watch  59263.jpg   
3  2011.0  Casual  Manchester United Men Solid Black Track Pants  21379.jpg   
4  2012.0  Casual                          Puma Men Grey T-shirt  53759.jpg   

  filename_y                                            

In [54]:
#Hapus yang tidak ada gambar
merge_data['file_found'] = merge_data['filename_x'].apply(lambda x: x in image_files)
merge_data = merge_data[merge_data['file_found']].reset_index(drop=True)
merge_data.drop(columns=['file_found'], inplace=True)
print("Filtered Merged DataFrame:")
print(merge_data.head())

Filtered Merged DataFrame:
      id gender masterCategory subCategory  articleType baseColour  season  \
0  15970    Men        Apparel     Topwear       Shirts  Navy Blue    Fall   
1  39386    Men        Apparel  Bottomwear        Jeans       Blue  Summer   
2  59263  Women    Accessories     Watches      Watches     Silver  Winter   
3  21379    Men        Apparel  Bottomwear  Track Pants      Black    Fall   
4  53759    Men        Apparel     Topwear      Tshirts       Grey  Summer   

     year   usage                             productDisplayName filename_x  \
0  2011.0  Casual               Turtle Check Men Navy Blue Shirt  15970.jpg   
1  2012.0  Casual             Peter England Men Party Blue Jeans  39386.jpg   
2  2016.0  Casual                       Titan Women Silver Watch  59263.jpg   
3  2011.0  Casual  Manchester United Men Solid Black Track Pants  21379.jpg   
4  2012.0  Casual                          Puma Men Grey T-shirt  53759.jpg   

  filename_y                 

TRAIN

In [55]:
data = merge_data


In [59]:
path = "../tmp/fashion-dataset/images"
img_size = 224
datagen = ImageDataGenerator(rescale=1./255.)
generator = datagen.flow_from_dataframe(
    dataframe=merge_data,
    directory=path,
    x_col='filename_x',
    target_size=(img_size, img_size),
    class_mode=None,
    batch_size=32,
    shuffle=False
)

Found 44072 validated image filenames.


In [ ]:
##MASIH ERROR
# Menentukan ukuran gambar
img_size = 224
# Menggunakan model InceptionV3
base_model = InceptionV3(include_top=False, input_shape=(img_size, img_size, 3))
# Membekukan semua layer pada base_model
for layer in base_model.layers:
    layer.trainable = False
# Membuat input layer
input_layer = Input(shape=(img_size, img_size, 3))
# Memasukkan input_layer ke base_model
x = base_model(input_layer)
# Menambahkan GlobalAveragePooling2D layer
output = GlobalAveragePooling2D()(x)
# Membuat model embeddings
embeddings = Model(inputs=input_layer, outputs=output)
# Menampilkan ringkasan model embeddings
embeddings.summary()
X = embeddings.predict(generator, verbose = 1)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d_2  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
Total params: 21802784 (83.17 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 21802784 (83.17 MB)
_________________________________________________________________
  64/1378 [>.............................] - ETA: 1:35:56

In [ ]:
#PCA
pca = PCA(2)
X_pca = pca.fit_transform(X)
styles[['pc1','pc2']] = X_pca
plt.figure(figsize = (20,8))
sns.scatterplot(x='pc1', y='pc2', data=styles_df, hue = 'masterCategory')
plt.show()

In [ ]:
def read_img(image_path):
    image = load_img(os.path.join(path,image_path),target_size=(img_size,img_size,3))
    image = img_to_array(image)
    image = image/255.
    return image

In [ ]:
y = styles_df['id']
nearest_neighbours = KNeighborsClassifier(n_neighbors = 7)
nearest_neighbours.fit(X,y)
KNeighborsClassifier(n_neighbors=7)
styles = styles.reset_index(drop=True)

for _ in range(10):
    i = random.randint(0,len(styles_df))
    img1 = read_img(styles_df.loc[i,'filename'])
    dist, index = nearest_neighbours.kneighbors(X=X[i,:].reshape(1,-1))
    plt.figure(figsize = (4 , 4))
    plt.imshow(img1)
    plt.title("Input Image")
    plt.axis('off')

    plt.figure(figsize = (20 , 20))
    for i in range(1,6):
        plt.subplot(1 , 5, i)
        plt.subplots_adjust(hspace = 0.5 , wspace = 0.3)
        image = read_img(styles_df.loc[index[0][i],'filename'])
        plt.imshow(image)
        plt.title(f'Similar Product #{i}')
        plt.axis('off')